In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import flatten, udf, col
from pyspark.sql.types import BooleanType
bucket = 'covidbucket531'
spark = SparkSession.builder\
    .appName("covid_project")\
    .config("spark.jars", "gs://spark-lib/bigquery/spark-bigquery-latest.jar")\
    .master('yarn')\
    .getOrCreate()

df = spark.read \
    .format("bigquery") \
    .load('charanfinalproject1.data531.covid')
df.show(10)

+-------------+----------+-----+------------+---+------------+---------+---+--------+--------+----+------+-------+------------+-------------+--------------+-------+-------------+-------+--------------------+-----+
|int64_field_0|Unnamed__0|USMER|MEDICAL_UNIT|SEX|PATIENT_TYPE|PNEUMONIA|AGE|PREGNANT|DIABETES|COPD|ASTHMA|INMSUPR|HIPERTENSION|OTHER_DISEASE|CARDIOVASCULAR|OBESITY|RENAL_CHRONIC|TOBACCO|CLASIFFICATION_FINAL|DEATH|
+-------------+----------+-----+------------+---+------------+---------+---+--------+--------+----+------+-------+------------+-------------+--------------+-------+-------------+-------+--------------------+-----+
|           48|        48|    0|           1|  1|           1|        0| 25|       0|       0|   0|     1|      0|           0|            0|             0|      0|            0|      0|                   7|    0|
|       297183|    297183|    0|           1|  1|           1|        0| 27|       0|       0|   0|     0|      0|           0|            0|   

In [2]:
Data  =df

In [3]:
from pyspark.sql.types import IntegerType,FloatType
Data2 = df.withColumn("USMER", Data["USMER"].cast(IntegerType()))\
            .withColumn("MEDICAL_UNIT", Data["MEDICAL_UNIT"].cast(IntegerType()))\
            .withColumn("SEX", Data["SEX"].cast(IntegerType()))\
            .withColumn("PATIENT_TYPE", Data["PATIENT_TYPE"].cast(IntegerType()))\
            .withColumn("PNEUMONIA", Data["PNEUMONIA"].cast(IntegerType()))\
            .withColumn("AGE", Data["AGE"].cast(IntegerType()))\
            .withColumn("PREGNANT", Data["PREGNANT"].cast(IntegerType()))\
            .withColumn("DIABETES", Data["DIABETES"].cast(IntegerType()))\
            .withColumn("COPD", Data["COPD"].cast(IntegerType()))\
            .withColumn("ASTHMA", Data["ASTHMA"].cast(IntegerType()))\
            .withColumn("INMSUPR", Data["INMSUPR"].cast(IntegerType()))\
            .withColumn("HIPERTENSION", Data["HIPERTENSION"].cast(IntegerType()))\
            .withColumn("OTHER_DISEASE", Data["OTHER_DISEASE"].cast(IntegerType()))\
            .withColumn("CARDIOVASCULAR", Data["CARDIOVASCULAR"].cast(IntegerType()))\
            .withColumn("OBESITY", Data["OBESITY"].cast(IntegerType()))\
            .withColumn("RENAL_CHRONIC", Data["RENAL_CHRONIC"].cast(IntegerType()))\
            .withColumn("TOBACCO", Data["TOBACCO"].cast(IntegerType()))\
            .withColumn("CLASIFFICATION_FINAL", Data["CLASIFFICATION_FINAL"].cast(IntegerType()))\
            .withColumn("DEATH", Data["DEATH"].cast(IntegerType()))


In [4]:
Data3 = Data2.drop("int64_field_0") 

In [5]:
Data4 = Data3.drop("Unnamed__0") 

In [6]:
Data4.printSchema()

root
 |-- USMER: integer (nullable = true)
 |-- MEDICAL_UNIT: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- PATIENT_TYPE: integer (nullable = true)
 |-- PNEUMONIA: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PREGNANT: integer (nullable = true)
 |-- DIABETES: integer (nullable = true)
 |-- COPD: integer (nullable = true)
 |-- ASTHMA: integer (nullable = true)
 |-- INMSUPR: integer (nullable = true)
 |-- HIPERTENSION: integer (nullable = true)
 |-- OTHER_DISEASE: integer (nullable = true)
 |-- CARDIOVASCULAR: integer (nullable = true)
 |-- OBESITY: integer (nullable = true)
 |-- RENAL_CHRONIC: integer (nullable = true)
 |-- TOBACCO: integer (nullable = true)
 |-- CLASIFFICATION_FINAL: integer (nullable = true)
 |-- DEATH: integer (nullable = true)



In [7]:
required_features = [
                    'USMER',
                    'MEDICAL_UNIT',
                    'SEX',
                    'PATIENT_TYPE',
                     'PNEUMONIA',
                     'AGE',
                     'PREGNANT',
                     'DIABETES',
                     'COPD',
                     'ASTHMA',
                     'INMSUPR',
                     'HIPERTENSION',
                     'OTHER_DISEASE',
                     'CARDIOVASCULAR',
                     'RENAL_CHRONIC',
                     'TOBACCO',
                     'CLASIFFICATION_FINAL',
                     'DEATH'
                   ]

In [8]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=required_features, outputCol='features')
transformed_data = assembler.transform(Data4)

In [9]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [10]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = 'features' , labelCol = 'DEATH', maxIter=5)
lrModel = lr.fit(training_data)

23/05/03 23:02:21 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/03 23:02:21 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [11]:
rf_predictions = lrModel.transform(test_data)

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'DEATH', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(rf_predictions))

Logistic Regression Accuracy: 0.9772444727883738


In [15]:
df = rf_predictions

In [19]:
tp = df[(df.DEATH == 1) & (df.prediction == 1)].count()
tn = df[(df.DEATH == 0) & (df.prediction == 0)].count()
fp = df[(df.DEATH == 0) & (df.prediction == 1)].count()
fn = df[(df.DEATH == 1) & (df.prediction == 0)].count()
print ("True Positives:", tp)
print ("True Negatives:", tn)
print ("False Positives:", fp)
print ("False Negatives:", fn)
print ("Total", df.count())

r = float(tp)/(tp + fn)
print ("recall", r)

p = float(tp) / (tp + fp)
print ("precision", p)

True Positives: 2158142
True Negatives: 2323317
False Positives: 56405
False Negatives: 47344


Total 4584989
recall 0.97853352957126
precision 0.9745297796795462


In [ ]:
confusion matrix :
    
[[2158142  47344
  56405.  2323317]]

In [ ]:
rf_predictions

In [26]:
# lrModel.save(spark,"gs://covid531/")

In [27]:
lrModel.write().overwrite().save("gs://covid531/model12.pkl")